In [1]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('orange_small_churn_data.txt')
labels = pd.read_csv('orange_small_churn_labels.txt', header=None)
# df_train.dropna(axis=1, how='all', inplace=True)
df['target'] = labels

In [7]:
df.shape

(40000, 231)

Отделим hold-out dataset

In [8]:
df_train, df_ho = train_test_split(df, test_size=5000, shuffle=True)

In [9]:
df_ho.to_csv('hold_out_dataset.csv')

Опишем способы, которыми можно обрабатывать категориальные признаки для подачи на вход модели

- one hot encoder: кодирование признаков в бинарные колонки; плохо подходит для моделей, основанных на деревьях
    - sklearn.preprocessing.OneHotEncoder
    - pandas.get_dummies
- label encoder: замена значений категориальных признаков на целочисленные номера; плохо подходит для линейных моделей, так как задаёт отношения порядка на признаке, то есть добавляет ложную информацию (если реально порядка нет)
    - sklearn.preprocessing.LabelEncoder
    - sklearn.preprocessing.OrdinalEncoder
- frequency encoding: кодируем значения частотой значения признака в колонке
- mean target encoding: значения заменяются средними значениями целевой переменной внутри каждоый категории
- Leave One Out Encoding: аналогично target encoding, но для вычисления среднего не используется текущая строка; приводит к уменьшению переобучения
- хеширование: похоже на ohe, но получается гораздо меньше новых признаков; подходит для признаков с большим количеством уникальных значений

Ключевой метрикой качества выберем ROC AUC.
- наша задача - бинарная классификация
- AUC не зависит от сбалансированности выборки
- метрика не зависит от выбора порога

Вспомогательные метрики:
- precision - чтобы понять ошибки какого рода допускает модель
- recall
- F1
- log loss с весами - чтобы оценить насколько сильно абсолютные значения предсказаний отличаются от истинных

После отделения hold-out датасета остаётся 35000 строк. Возьмём количество фолдов равным 7. Тогда на кросс-валидации выборка делится в отношении 1 / 6.

Стратегия кросс-валидации:
- stratified k-fold: выборка несбалансирована, поэтому применим стратификацию
- shuffle: перемешиваем выборку для равномерного распределения таргета по фолдам